# CardioIA — Fase 3: Séries Temporais
Comparação simples de métodos de detecção de anomalias em BPM/Temperatura.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(42)
n = 600
t = np.arange(n)
bpm = 72 + 3*np.sin(2*np.pi*t/120) + np.random.normal(0, 1.5, n)
temp = 36.6 + 0.1*np.sin(2*np.pi*t/200) + np.random.normal(0, 0.05, n)
# Injeta eventos
events = np.random.choice(n, size=8, replace=False)
bpm[events] = 125
temp[events[:4]] = 38.6
df = pd.DataFrame({'bpm': bpm, 'temp': temp})
df.head()

In [ ]:
# Método 1: Z-score
def zscore_anom(x, thr=3.0):
    z = (x - x.mean()) / x.std(ddof=0)
    return (np.abs(z) > thr)
df['anom_bpm_z'] = zscore_anom(df['bpm'], thr=3.0)
df['anom_temp_z'] = zscore_anom(df['temp'], thr=3.0)
df[['anom_bpm_z','anom_temp_z']].mean()

In [ ]:
# Método 2: Desvio absoluto mediano (MAD)
def mad_anom(x, thr=3.5):
    med = np.median(x)
    mad = np.median(np.abs(x - med)) + 1e-6
    score = 0.6745 * (x - med) / mad
    return (np.abs(score) > thr)
df['anom_bpm_mad'] = mad_anom(df['bpm'])
df['anom_temp_mad'] = mad_anom(df['temp'])
df[['anom_bpm_mad','anom_temp_mad']].mean()

In [ ]:
# Visualização rápida
fig, ax = plt.subplots(2,1, figsize=(10,6), sharex=True)
ax[0].plot(df['bpm'], label='bpm')
ax[0].plot(df.index[df['anom_bpm_mad']], df['bpm'][df['anom_bpm_mad']], 'rx', label='anom MAD')
ax[0].axhline(120, color='r', ls='--', alpha=0.3)
ax[0].legend()
ax[1].plot(df['temp'], label='temp')
ax[1].plot(df.index[df['anom_temp_mad']], df['temp'][df['anom_temp_mad']], 'rx', label='anom MAD')
ax[1].axhline(38, color='r', ls='--', alpha=0.3)
ax[1].legend()
plt.tight_layout()
plt.show()

## Observações
- Z-score e MAD funcionam bem para séries estáveis e destacam picos.
- Em produção, considerar janelas móveis, sazonalidade e modelos específicos (ARIMA, LSTM, Prophet).